## Configuration and Setup


In [ ]:
import os
os.environ['HF_HOME'] = "D:/HF_CACHE/"

In [ ]:
from cs324_project.datasets import GlueDatasetTask, load_glue_dataset_info
from cs324_project.models import ModelCheckpointName, load_classification_model, load_pretraining_model, load_tokenizer
from cs324_project.training import get_training_args, get_trainer_mlm, get_trainer_sc, get_latest_checkpoint_path
from cs324_project.masking import get_whole_word_masking_data_collator, get_random_masking_data_collator, default_data_collator

In [ ]:
model_name = ModelCheckpointName.TINYBERT_HUAWEI
task = GlueDatasetTask.COLA

In [ ]:
tokenizer = load_tokenizer(model_name)
dataset_info = load_glue_dataset_info(task, tokenizer, reduce_fraction=0.1)

## Fine-tune with masking

In [ ]:
model_mlm = load_pretraining_model(model_name, dataset_info)

In [ ]:
training_args = get_training_args(task, num_epochs=1)
data_collator = get_random_masking_data_collator(tokenizer)
trainer_mlm = get_trainer_mlm(
    dataset_info=dataset_info,
    model=model_mlm,
    training_args=training_args,
    data_collator=data_collator)

In [ ]:
trainer_mlm.train()

In [ ]:
trainer_mlm.evaluate()

In [ ]:
checkpoint_path = get_latest_checkpoint_path(training_args)

## Fine-tune on sequence classification task

In [ ]:
model_sc = load_classification_model(checkpoint_path, dataset_info)

In [ ]:
training_args = get_training_args(task, num_epochs=1)
trainer = get_trainer(dataset_info, model_sc, training_args)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()